In [3]:
import pandas as pd
from time import sleep
import os
from selenium import webdriver # for interacting with website
from selenium.webdriver.common.by import By
import re

In [5]:
def transcript2df(transcript):
    pattern = r'^(([0-9]{2}|[0-9]{1}):([0-9]{2}|[0-9]{1})|([0-9]{2}|[0-9]{1})):[0-9]{2}$'
    transcript_timestamps,transcript_dialog,prev = [],[],None
    for item in transcript:
        if re.match(pattern, item):
            transcript_timestamps.append(item)
            if prev == None: prev = item
            else: 
                transcript_dialog.append(prev)
                prev = item
        else:
            if re.match(pattern, prev):
                prev = item
            else:
                prev = prev + ' ' + item
    transcript_dialog.append(item)
    df = pd.DataFrame({'timestamp':transcript_timestamps, 
                   'text':transcript_dialog})    
    return df

In [36]:
def get_youtube_transcript(video_ids):
    frames = []
    driver = webdriver.Chrome()
    for video_id in video_ids:
        try:
            print(video_id)
            driver.get(f"https://www.youtube.com/watch?v={video_id}")
            sleep(4)
            driver.find_element(By.XPATH,'//*[@id="button-shape"]/button').click()
            sleep(2)
            driver.find_element(By.XPATH,'//*[@id="items"]/ytd-menu-service-item-renderer[2]/tp-yt-paper-item/yt-formatted-string').click()
            sleep(4)
            transcript_element = driver.find_element(By.XPATH,"//*[@id='body']/ytd-transcript-segment-list-renderer")
            transcript = transcript_element.text
            transcript = transcript.split('\n')
            transcript.pop(0)
            frames.append(transcript2df(transcript).assign(video_id=video_id))
        except:
            print(video_id)
    driver.quit()
    return pd.concat(frames,ignore_index=True)

In [4]:
df = pd.read_parquet('../scripts/data/scrap_oso_trava/playlist_videos.parquet')

In [38]:
result = get_youtube_transcript(df.id.unique()[:5])
result

fL9FNpuGK_8    1139
OGwMD9NMQMA    1015
uTS8L1kvjmY     942
027NOppIrVQ     919
2u6FeKJ-biw     896
Name: video_id, dtype: int64